In [1]:
from numba import njit
import scipy.stats as stats
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('..', '..')))
from src import *
from itertools import product

import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


In [2]:
L = 1000
ps = np.array([0.5, 0.7, 0.9, 1.0, 1.2, 1.4, 1.7, 2.0, 3.0, 5.0])
ps_mask = [True, True, True, True, True, True, True, True, True, False]
realizations = 1000

data = np.load("../../data/wave_function_day_data.npy")

In [3]:
def produce_centred_eigenvector(p, L):
    ness_correlation = C_NESS(0, 0, A(L, p))
    _, eigenvectors = np.linalg.eig(ness_correlation)
    for i, arr in enumerate(eigenvectors):
        central_index = np.argmax(arr)
        if 2.4*L/5 <= central_index <= 2.6*L/5:
            return np.abs(arr)**2

low_p_eigenvector = [produce_centred_eigenvector(0.5, 1000) for i in range(5)]
high_p_eigenvector = [produce_centred_eigenvector(1.1, 1000) for i in range(5)]


In [4]:
fit_lower_bound = 10
fit_upper_bound = int(2*L/5)
x = np.abs(np.arange(L))
grads = np.zeros(len(data))
y_intercepts = np.zeros(len(data))
for i, y in enumerate(data):
    g, c = fit_log_log(x[x <= fit_upper_bound], y[x <= fit_upper_bound], np.arange(
        fit_lower_bound, fit_upper_bound))
    grads[i] = -g
    y_intercepts[i] = c

In [9]:
%matplotlib qt

# Plotting parameters
scaling = 1.0
title_font_size = 10*scaling
plt.rcParams['xtick.major.pad'] = '0'
plt.rcParams['ytick.major.pad'] = '0'
plt.rcParams.update({
    "font.family": "Times New Roman",
    "font.size": 10*scaling,
    "text.usetex": True,
})

fit_lower_bound = 10
fit_upper_bound = int(2*L/5)

pt = 0.0138889
fig, ax = plt.subplots(
    nrows=3, ncols=1, figsize=(246*pt*scaling, 250*1.3*pt*scaling),
    gridspec_kw={
    'left': 0.175, 'right': 0.99, 'top': 0.985, 'bottom': 0.105, 'hspace': 0.2,
    'height_ratios': (0.25, 0.25, 0.5)
})

for arr in low_p_eigenvector:
    ax[0].plot(arr)
ax[0].set_ylabel(r"$|\psi(i)|^2$", labelpad = -2)
ax[0].text(-0.2, 0.95, r"$\mathrm{(a)}$", transform = ax[0].transAxes)
ax[0].set_xlabel(r"$i$", labelpad=-8)
#ax[0][0].set_yscale('log')
#ax[0][0].set_xscale('log')

for arr in high_p_eigenvector:
    ax[1].plot(arr)
ax[1].set_ylabel(r"$|\psi(i)|^2$")
ax[1].text(-0.2, 0.95, r"$\mathrm{(b)}$", transform=ax[1].transAxes)
ax[1].set_xlabel(r"$i$", labelpad=-8)
#ax[0][1].set_yscale('log')
#ax[0][1].set_xscale('log')

x = np.abs(np.arange(L))
for i in [0, 1, 2, 3, 4, 5, 6, 7]:
    if i in range(len(ps)-2):
        ax[2].plot(x[x <= fit_upper_bound], data[i][x <= fit_upper_bound], linewidth=2.5, label=r"$p = {}, a = {}, c = {}$".format(
            ps[i], round(grads[i], 3), round(math.exp(y_intercepts[i])), 3))
        linear_x_axis = np.arange(fit_lower_bound, fit_upper_bound)
        ax[2].plot(linear_x_axis, math.exp(y_intercepts[i])*(linear_x_axis**(-grads[i])),
                   color='k', lw=1, ls="--")
ax[2].set_xlabel(r"$|i-i_\mathrm{max}|$")
ax[2].set_ylabel(r"$\left\langle|\psi(i)|^2\right\rangle$")
ax[2].set_yscale('log')
ax[2].set_xscale('log')
ax[2].set_ylim(10**(-6), 10**(-2))
ax[2].text(-0.2, 0.95, r"$\mathrm{(c)}$", transform = ax[2].transAxes)

inset_ax = inset_axes(ax[2], width="50%", height="55%", loc=3,
                   bbox_to_anchor=(.07, .075, 0.95, 0.8), bbox_transform=ax[2].transAxes)
inset_ax.set_xlabel(r"$p$", labelpad=-9*scaling)
inset_ax.plot(ps[ps_mask], grads[ps_mask], label=r"$y(p)=a(p)$")
inset_ax.plot(ps[ps_mask], ps[ps_mask], label=r"$y(p)=p$", ls="--", color="k")


ax[2].annotate("p", xy=(250, 10**(-3)), xytext=(50, 10**(-5.5)),
               arrowprops=dict(arrowstyle="<|-", facecolor="black", lw=1.5),
               bbox=dict(pad=-1, facecolor="none", edgecolor="none"))

plt.show()


In [155]:
plt.savefig("../../paper_figs/wave_function_decay.eps")